In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings('ignore')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/zomato-bangalore-restaurants/zomato.csv")

In [ ]:
df.head()

In [ ]:
df.isna().sum()

In [ ]:
df.nunique()

In [ ]:
df['location'].unique()

In [ ]:
df['cuisines'].unique()

In [ ]:
df['name'].unique()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
zom=df.drop(['url','dish_liked','phone','location'],axis=1)

In [ ]:
df['rate'].unique()

In [ ]:
df['votes'].unique()


In [ ]:
zom.head()

In [ ]:
zom.shape

In [ ]:
zom.dropna(how="any",inplace=True)

In [ ]:
zom.info()

In [ ]:
zom.isna().sum()

In [ ]:
zom.shape

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
zom.profile_report()

In [ ]:
zom.duplicated().sum()

In [ ]:
zom.drop_duplicates(inplace=True)
zom.head()

In [ ]:
zom.shape

In [ ]:
#Changing the column names
zom = zom.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type',
                                  'listed_in(city)':'city'})
zom.columns

In [ ]:
#Some Transformations
zom['cost'] = zom['cost'].astype(str) #Changing the cost to string
zom['cost'] = zom['cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zom['cost'] = zom['cost'].astype(float) # Changing the cost to Float
zom.info() # looking at the dataset information after transformation

In [ ]:
zom.head()

In [ ]:
#Removing '/5' from Rates
zom = zom.loc[zom.rate !='NEW']
zom = zom.loc[zom.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zom.rate = zom.rate.apply(remove_slash).str.strip().astype('float')
zom['rate'].head() # looking at the dataset after transformation

In [ ]:
corelation=zom.corr()

In [ ]:
sns.heatmap(corelation,xticklabels=corelation.columns,yticklabels=corelation.columns,annot=True,cmap='coolwarm')

In [ ]:
# Top rated hotels (Revieved by at least 3000 people)
data_order = zom.groupby(['name', 'cost', 'cuisines'], as_index=False)[['votes', 'rate']].mean()
data_order = data_order[data_order['votes']>7127]
data_order = data_order.sort_values('votes', ascending=False).head(30)
data_order

In [ ]:
g=sns.barplot(x = data_order['name'],
            y = data_order['votes'],alpha=0.8)
plt.xticks(rotation=90)                 
plt.title("Top 10 most reviewed hotels")

In [ ]:
#Restaurants delivering Online or not
sns.countplot(zom['online_order'])
fig = plt.gcf()
fig.set_size_inches(10,8)
plt.title('Restaurants delivering online or Not')

In [ ]:
sns.countplot(zom['book_table'])
fig = plt.gcf()
fig.set_size_inches(10,8)
plt.title('Restaurants allowing table booking or not')

In [ ]:
g = sns.countplot(x="city",data=zom, palette = "Set1")
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
g 
plt.title("locality",size = 20)

In [ ]:
restype=zom['type'].value_counts().sort_values(ascending=True)
labels=['Pubs and bars','Buffet','Drinks & nightlife','Cafes','Desserts','Dine-out','Delivery ']
plt.title("Percentage of Restaurants according to their Type")
plt.pie(restype,labels=labels)